## Create I/O Data

In [ ]:
from utils.labels_utils import get_all_files_mhd
all_files_mhd = get_all_files_mhd()
len(all_files_mhd)

In [8]:
from utils.type_reader import mha_read_header
from utils.labels_utils import get_labels
import numpy as np
from tqdm import tqdm

# merge all images in one huge array

all_frames_filenames_array = np.empty((1))
frame_nums = np.empty((1))
labels = np.empty((1, 3))
i = 0
for f in all_files_mhd:
    if (i%20 == 0):
        print("File "+str(i)+"...")
    info = mha_read_header(f)
    labels = np.concatenate((labels, get_labels(f, info)), axis = 0)
    all_frames_filenames_array = np.concatenate((all_frames_filenames_array, [f]*info['Dimensions'][3]), axis=0)
    frame_nums = np.concatenate((frame_nums, np.arange(0, info['Dimensions'][3])), axis=0)
    i+=1
    
all_frames_filenames_array = all_frames_filenames_array[ 1:]
labels = labels[1:, :]
frame_nums = frame_nums[1:]

File 0...
File 20...
File 40...
File 60...
File 80...
File 100...
File 120...
File 140...
File 160...
File 180...
File 200...
File 220...
File 240...
File 260...
File 280...


In [9]:
X = np.vstack((all_frames_filenames_array, frame_nums)).transpose()
y = labels
print(X[0, :])
print(y[0, :])

['/data/Riabova/train3/0_22_5_0_0_0_3_1_1686308127391259.mhd' '0.0']
[ 91.57465921  94.48928267 106.19458738]


In [12]:
y.shape[0]

17717

In [10]:
from miscnn.data_loading.interfaces import Dictionary_interface
from utils.ImageDataset import getDatapointResized
from miscnn import Data_IO
import numpy as np

# Create 2D testing data set (grayscale image with binary mask)
dataset3D = dict()
for i in range(500): # range(0, y.shape[0]):
    img, mask, us_tip_coords_resized = getDatapointResized(X[i, 0], X[i, 1], y[i], mask_diam=3, resizeTo=128)
    dataset3D["TEST.sample_" + str(i)] = (img, mask)
    
# Initialize Dictionary IO Interface
io_interface3D = Dictionary_interface(dataset3D, channels = 1, classes=2, three_dim=True)

# Create the Data I/O object
data_io = Data_IO(io_interface3D, input_path="")